In [ ]:
import joblib
import time
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from numpy                 import array
from sklearn               import metrics
from sklearn.preprocessing import MinMaxScaler
from keras.models          import Sequential
from keras.layers          import *
from sklearn               import metrics
from tensorflow.keras import layers
from tcn import TCN
import keras
from keras.layers import Conv1D, MaxPooling1D, Dense, Flatten,Conv2D, MaxPool2D,LSTM,Bidirectional
from keras.models import Sequential
from keras import Input
from keras.layers          import *
from keras.models import save_model,load_model
from numpy import save,load
from sklearn.model_selection import KFold
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 忽略 INFO 和 WARNING 信息
import logging
tf.get_logger().setLevel(logging.ERROR)

In [ ]:
from dscribe.descriptors import SOAP
from dscribe.descriptors import MBTR,ACSF,CoulombMatrix,EwaldSumMatrix,SineMatrix 

In [ ]:
from pymatgen.core.surface import Slab
from pymatgen.io.ase import AseAtomsAdaptor
import ase.io

In [ ]:
#####decribe parameters#####
rcut = 6
#####decribe parameters#####
sm = SineMatrix(
    n_atoms_max=96,
    permutation="sorted_l2",
    sparse=False,
    flatten=False)
ew=EwaldSumMatrix(
    n_atoms_max=96,
    permutation="none",flatten=False,sparse=False)
acsf=ACSF(species=['Na','Mg','K','Ca','Sc','Ti','V','Cr','Mn','Fe','Co','Ni','Cu','Zn','Rb','Sr','Y','Zr','Nb','Mo','Tc','Rh','Pd','Ag','Cd','Cs','Ba','Os','Re','Ir','Pt','Au','Ga', 'In', 'Sn', 'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta', 'Hg', 'Tl', 'Pb', 'Bi', 'Li', 'Al','Ru','O'],
# acsf=ACSF(species=['Na','Mg','K','Ca','Sc','Ti','V','Cr','Mn','Fe','Co','Ni','Cu','Zn','Rb','Sr','Y','Zr','Nb','Mo','Tc','Rh','Pd','Ag','Cd','Cs','Ba','Os','Re','Ir','Pt','Au','Ru','O'],
    rcut=rcut,
    periodic=True,
    g2_params=[[1, 1], [1, 2], [1, 3]],
    g4_params=[[1, 1, 1], [1, 2, 1], [1, 1, -1], [1, 2, -1]])

In [ ]:
#encoding:utf-8
my_key='BJU2qDd4gpCPO0rf'
from pymatgen.core import Structure, Lattice
from pymatgen.core import Molecule
from pymatgen.analysis.adsorption import *
from pymatgen.core.surface import generate_all_slabs
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from matplotlib import pyplot as plt
from pymatgen.ext.matproj import MPRester
from pymatgen.io.vasp.inputs import Poscar
from pymatgen.io.vasp.inputs import Kpoints
from pymatgen.io.vasp.inputs import Incar
from pymatgen.io.vasp.inputs import Potcar
from pymatgen.io.vasp.inputs import PotcarSingle
from numpy import array
from pymatgen.io.vasp.sets import MPRelaxSet
from pymatgen.ext.matproj import MPRester
import re
import numpy as np
def sum_all(object_list):
    sum_result=0
    for strings in object_list:
        b=int(strings)
        sum_result+=b
    return sum_result
import random
m= MPRester(my_key)
element_list=['K','Ca','Sc','Ti','V','Cr','Mn','Fe','Co','Ni','Rb','Sr','Y','Zr','Nb','Mo','Tc','Rh','Cs','Ba','Os','Re','Ir']
extra_element_list=['Ga', 'In', 'Sn', 'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta', 'Hg', 'Tl', 'Pb', 'Bi', 'Li', 'Al']


RuO2=m.get_structure_by_material_id('mp-825')
struct = SpacegroupAnalyzer(RuO2).get_conventional_standard_structure()
struct=reorient_z(struct)
slabs = generate_all_slabs(struct, 1,12.0, 15.0, center_slab=True)
m_index=(1,1,0)
slab_dict = {slab.miller_index:slab for slab in slabs}                   
RuO2_110=slab_dict[m_index]
RuO2_110.make_supercell([[2,0,0],
                          [0,2,0],
                          [0,0,1]])

In [ ]:
def convert_crystal_to_des(ase_atoms):
    des1=sm.create(ase_atoms,n_jobs=-1)
    des2=ew.create(ase_atoms,n_jobs=-1)
    des3=acsf.create(ase_atoms,n_jobs=-1)
    new_des=np.concatenate((des1,des2),axis=1)
    new_des=np.concatenate((new_des,des3),axis=1)
    new_des=new_des.reshape(768,969)
    return new_des

In [ ]:
##dscribe 1.2.1 numpy 1.23.5
def load_trained_domain_adapted_model(model_type, saved_model_path):
    # Load the model
    if model_type in [7, 8]:  # For TCN models
        #laptop running have to change to complie=False and tf....
        current_model = tf.keras.models.load_model(saved_model_path, custom_objects={'TCN': TCN},compile=False)
    else:
        current_model = tf.keras.models.load_model(saved_model_path,compile=False)
    return current_model

In [ ]:
def predict_single_model(new_des, loaded_model, INPUT_SCALER, OUTPUT_SCALER, model_type):
    # 对新晶体数据进行归一化
    new_des_normalized = INPUT_SCALER.transform(new_des.reshape(-1, new_des.shape[-1])).reshape(new_des.shape)

    # 使用模型进行预测
    predicted = loaded_model.predict(new_des_normalized.reshape(1, *new_des_normalized.shape),verbose=0)

    # 特殊处理模型类型6
    if model_type == 6:
        # 确保预测结果的形状正确
        predicted = predicted.reshape(-1, 1)
    # 逆归一化预测结果
    predicted_original = OUTPUT_SCALER.inverse_transform(predicted)
    return predicted_original

In [ ]:
loaded_model_3=load_trained_domain_adapted_model(3,"Domain_Adapted_SLAB_3.h5")
loaded_model_5=load_trained_domain_adapted_model(5,"Domain_Adapted_SLAB_5.h5")
loaded_model_6=load_trained_domain_adapted_model(6,"Domain_Adapted_SLAB_6.h5")
loaded_model_7=load_trained_domain_adapted_model(7,"Domain_Adapted_SLAB_7.h5")
loaded_model_9=load_trained_domain_adapted_model(9,"Domain_Adapted_SLAB_9.h5")
SLAB_INPUT_SCALER_B=joblib.load("SLAB_INPUT_SCALER_B.pkl")
SLAB_OUTPUT_SCALER_B=joblib.load("SLAB_OUTPUT_SCALER_B.pkl")

In [ ]:
def calculate_variance(new_des):
    predictions = [
        predict_single_model(new_des, loaded_model_3, SLAB_INPUT_SCALER_B, SLAB_OUTPUT_SCALER_B, 3)[0][0],
        predict_single_model(new_des, loaded_model_5, SLAB_INPUT_SCALER_B, SLAB_OUTPUT_SCALER_B, 5)[0][0],
        predict_single_model(new_des, loaded_model_6, SLAB_INPUT_SCALER_B, SLAB_OUTPUT_SCALER_B, 6)[0][0],
        predict_single_model(new_des, loaded_model_7, SLAB_INPUT_SCALER_B, SLAB_OUTPUT_SCALER_B, 7)[0][0],
        predict_single_model(new_des, loaded_model_9, SLAB_INPUT_SCALER_B, SLAB_OUTPUT_SCALER_B, 9)[0][0]
    ]
    variance = np.var(predictions)
    return variance


In [ ]:
def calculate_weighted_energy(new_des):
    #0.9950695604822077
    predict_3=predict_single_model(new_des,loaded_model_3,SLAB_INPUT_SCALER_B,SLAB_OUTPUT_SCALER_B,3)[0][0]
    #0.9965353226220658
    predict_5=predict_single_model(new_des,loaded_model_5,SLAB_INPUT_SCALER_B,SLAB_OUTPUT_SCALER_B,5)[0][0]
    #0.996410743935558
    predict_6=predict_single_model(new_des,loaded_model_6,SLAB_INPUT_SCALER_B,SLAB_OUTPUT_SCALER_B,6)[0][0]
    #0.9938711994655637
    predict_7=predict_single_model(new_des,loaded_model_7,SLAB_INPUT_SCALER_B,SLAB_OUTPUT_SCALER_B,7)[0][0]
    #0.9973437346125353
    predict_9=predict_single_model(new_des,loaded_model_9,SLAB_INPUT_SCALER_B,SLAB_OUTPUT_SCALER_B,9)[0][0]
    sum_weight=0.9950695604822077+0.9965353226220658+0.996410743935558+0.9938711994655637+0.9973437346125353
    sum_result=0.9950695604822077*predict_3+0.9965353226220658*predict_5+0.996410743935558*predict_6+0.9938711994655637*predict_7+0.9973437346125353*predict_9
    return sum_result/sum_weight

In [ ]:
ase_atoms = AseAtomsAdaptor.get_atoms(RuO2_110)
base_des_ruo2=convert_crystal_to_des(ase_atoms)

In [ ]:
ruo2_slab_predict=calculate_weighted_energy(base_des_ruo2)
print(ruo2_slab_predict)

In [ ]:
tf.get_logger().setLevel(logging.ERROR)

In [ ]:
import random
from pymatgen.core.surface import Slab
from pymatgen.core.periodic_table import Element
from deap import base, creator, tools, algorithms

def calculate_energy(new_structure):
    ase_atoms = AseAtomsAdaptor.get_atoms(new_structure)
    new_des=convert_crystal_to_des(ase_atoms)
    weighted_energy_final=calculate_weighted_energy(new_des)
#     print(weighted_energy_final)
    return weighted_energy_final

# 定义适应度函数
def evaluate(individual):
    # 根据个体（染色体）创建新的晶体结构
    new_structure = RuO2_110.copy()
    dopants = ['Ca', 'Ca', 'Ca', 'Mn', 'Mn', 'Pr']  # 根据配方
    for i, site in enumerate(individual):
        new_structure.replace(site, dopants[i])
    # 计算并返回能量
    energy = calculate_energy(new_structure)
    return (energy,)  # 注意这里返回的是一个元组

# 遗传算法参数
NUM_SITES = 32  # 晶格中的Ru原子数量
NUM_DOPANTS = 6  # 掺杂原子总数

# 创建不重复的个体
def create_individual():
    return random.sample(range(NUM_SITES), NUM_DOPANTS)
# 自定义交叉操作
def cxTwoPointCopy(ind1, ind2):
    tools.cxTwoPoint(ind1, ind2)
    # 修正重复元素
    fix_duplicate(ind1)
    fix_duplicate(ind2)
    return ind1, ind2

# 自定义变异操作
def mutShuffleIndexes(individual, indpb):
    tools.mutShuffleIndexes(individual, indpb)
    # 修正重复元素
    fix_duplicate(individual)
    return individual,

# 修正重复元素的函数
def fix_duplicate(individual):
    unique_sites = set(individual)
    for i in range(len(individual)):
        while individual[i] in unique_sites:
            individual[i] = random.randrange(NUM_SITES)
        unique_sites.add(individual[i])

# 创建适应度类和个体类
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # 最小化问题
creator.create("Individual", list, fitness=creator.FitnessMin)

# 初始化工具箱
toolbox = base.Toolbox()
toolbox.register("indices", random.sample, range(NUM_SITES), NUM_DOPANTS)
toolbox.register("individual", tools.initIterate, creator.Individual, create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate)

toolbox.register("mate", cxTwoPointCopy)
toolbox.register("mutate", mutShuffleIndexes, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

# 运行遗传算法
def run_genetic_algorithm():
    pop = toolbox.population(n=500)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("min", lambda x: min(v[0] for v in x))  # 修改最小值统计
    stats.register("avg", lambda x: sum(v[0] for v in x) / len(x))  # 修改平均值统计

    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=20, 
                                   stats=stats, halloffame=hof, verbose=True)

    return hof[0]





In [ ]:
for run_repeat_time in range(1,26):
    start_time = time.time()
    best_structure = run_genetic_algorithm()
    end_time = time.time()
    print("best_doping_structure", best_structure,"monte_carlo times:",run_repeat_time)
    print(f"Completed in {end_time - start_time:.2f} seconds")
    run_repeat_time+=1